In [4]:
import pandas as pd

from generate_cases import generate_views, view_to_natural_language
from pyetr.inference import (
    default_inference_procedure,
    classically_valid_inference_procedure,
)

In [12]:
views = generate_views(
    n_views=1000,
    max_conjuncts=3,
    max_disjuncts=3,
    generate_supposition=True,
    neg_prob=0.2,
)

In [13]:
from pyetr.view import View
def generate_premises(views: list[View]) -> list[tuple[View, ...]]:
    ret = []
    for v1 in views:
        for v2 in views:
            if len(v1.atoms.intersection(v2.atoms)) > 1:
                ret.append((v1, v2))
    return ret

In [14]:
premises = generate_premises(views)

In [15]:
len(premises)

32998

In [3]:
# Get all ordered pairs of views -- too naive
# premises = [(v1, v2) for v1 in views for v2 in views]

In [16]:
df = pd.DataFrame(premises, columns=["p1", "p2"])

In [17]:
df.head()

,p1,p2
0,"{Eight()Three()One(),Seven()Jack()}^{Six()~Eig...","{Eight()Three()One(),Seven()Jack()}^{Six()~Eig..."
1,"{Eight()Three()One(),Seven()Jack()}^{Six()~Eig...",{Six()Seven()}^{}
2,"{Eight()Three()One(),Seven()Jack()}^{Six()~Eig...","{One(),Nine(),Seven()}^{Nine()Four(),One()Four()}"
3,"{Eight()Three()One(),Seven()Jack()}^{Six()~Eig...","{Nine()Two(),Eight()Jack()}^{Three()Five(),Two..."
4,"{Eight()Three()One(),Seven()Jack()}^{Six()~Eig...","{Five(),~King(),Three()Seven()}^{}"


In [18]:
df["p1_nl"] = df["p1"].apply(view_to_natural_language)
df["p2_nl"] = df["p2"].apply(view_to_natural_language)

In [19]:
df.head()

,p1,p2,p1_nl,p2_nl
0,"{Eight()Three()One(),Seven()Jack()}^{Six()~Eig...","{Eight()Three()One(),Seven()Jack()}^{Six()~Eig...",if either there is a six and not an eight and ...,if either there is a six and not an eight and ...
1,"{Eight()Three()One(),Seven()Jack()}^{Six()~Eig...",{Six()Seven()}^{},if either there is a six and not an eight and ...,there is a six and a seven
2,"{Eight()Three()One(),Seven()Jack()}^{Six()~Eig...","{One(),Nine(),Seven()}^{Nine()Four(),One()Four()}",if either there is a six and not an eight and ...,"if either there is a nine and a four, or there..."
3,"{Eight()Three()One(),Seven()Jack()}^{Six()~Eig...","{Nine()Two(),Eight()Jack()}^{Three()Five(),Two...",if either there is a six and not an eight and ...,"if either there is a three and a five, or ther..."
4,"{Eight()Three()One(),Seven()Jack()}^{Six()~Eig...","{Five(),~King(),Three()Seven()}^{}",if either there is a six and not an eight and ...,"either there is a five, or there is not a king..."


In [20]:
len(df)

32998

In [21]:
from tqdm.notebook import tqdm
tqdm.pandas()

df["etr_c"] = df.progress_apply(lambda row: default_inference_procedure([row["p1"], row["p2"]]), axis=1)

  0%|          | 0/32998 [00:00<?, ?it/s]

In [22]:
df.head()

,p1,p2,p1_nl,p2_nl,etr_c
0,"{Eight()Three()One(),Seven()Jack()}^{Six()~Eig...","{Eight()Three()One(),Seven()Jack()}^{Six()~Eig...",if either there is a six and not an eight and ...,if either there is a six and not an eight and ...,"{0,One()Eight()Three()Seven()Jack()}"
1,"{Eight()Three()One(),Seven()Jack()}^{Six()~Eig...",{Six()Seven()}^{},if either there is a six and not an eight and ...,there is a six and a seven,"{0,Six()Seven()}"
2,"{Eight()Three()One(),Seven()Jack()}^{Six()~Eig...","{One(),Nine(),Seven()}^{Nine()Four(),One()Four()}",if either there is a six and not an eight and ...,"if either there is a nine and a four, or there...","{0,~Four()~One(),~Four()~Nine()}"
3,"{Eight()Three()One(),Seven()Jack()}^{Six()~Eig...","{Nine()Two(),Eight()Jack()}^{Three()Five(),Two...",if either there is a six and not an eight and ...,"if either there is a three and a five, or ther...","{0,Three()One(),Seven()}"
4,"{Eight()Three()One(),Seven()Jack()}^{Six()~Eig...","{Five(),~King(),Three()Seven()}^{}",if either there is a six and not an eight and ...,"either there is a five, or there is not a king...","{0,Five(),~King(),Three()Seven()}"


In [23]:
cat_c_df = df[df["etr_c"].apply(lambda x: len(x.stage) == 1 and not x.is_verum)]
cat_c_df[
    (cat_c_df["p2"].apply(lambda x: len(x.stage) > 1)) &
    (cat_c_df["p1"].apply(lambda x: len(x.stage) > 1))
]

,p1,p2,p1_nl,p2_nl,etr_c
